## Preprocessing des Datensatzes


### Laden der benötigten Bibliotheken

In [ ]:
import pandas as pd
import numpy as np

import os
import glob

# Geofencing Bibliotheken
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

### Einlesen des Datensatzes
Dateiname des Datensatzes: Maschen_211207_TUDA_data.csv

In [ ]:
# Pfad zur CSV Datei 
pathMaschen = os.path.join("..", "..", "data", "Maschen_211207_TUDA_data.csv")
# importieren des Datensatzes in einen pd DataFrame
dfMaschen = pd.read_csv(pathMaschen)
dfMaschen = dfMaschen.sort_values(by=['wagon_ID','timestamp_measure_position'])
#dfMaschen = dfMaschen.head(1000000)

### Formatieren des Datensatzes / Preprocessing

In [ ]:
# Zeitpunkte aus Strings in Sekunden formatieren
def format_timestamp(timestamp_string: str) -> float:
    """
    Funktion, die aus dem timestamp_string einen float in sekunden zurückgibt
    :param timestamp_string: Zeitstempel
    :return: float: Zeit in Sekunden
    """
    string_list_leerzeichen = timestamp_string.split(' ')
    days = float(string_list_leerzeichen[0])
    days_in_sec = days * 24 * 60 * 60
    string_list_doppelpunkt = string_list_leerzeichen[-1].split(':')
    hours = float(string_list_doppelpunkt[0])
    hours_in_sec = hours * 60 * 60
    minutes = float(string_list_doppelpunkt[1])
    minutes_in_sec = minutes * 60
    sec_in_sec = float(string_list_doppelpunkt[2])
    total_time_in_sec = days_in_sec + hours_in_sec + minutes_in_sec + sec_in_sec
    return total_time_in_sec

In [ ]:
len(dfMaschen)

In [ ]:
# timestamp_measure_position mit NaN wird bereinigt (Nat in dieser Csv für timestmap_measure_position nicht vorhanden)
dfMaschen.drop(dfMaschen.loc[dfMaschen['timestamp_measure_position']=='NaT'].index, inplace=True)
print(len(dfMaschen))

# Anwenden der Funktion format_timestamp
dfMaschen['timestamp_measure_position'] = dfMaschen['timestamp_measure_position'].apply(format_timestamp) 

# Codieren des "movement_state" in integer: "moving"=2, "standing"=1, "parking"=0
# Codieren als String Operations
dfMaschen['movement_state'].replace('moving', '2', inplace=True, regex=True)
dfMaschen['movement_state'].replace('standing', '1', inplace=True, regex=True)
dfMaschen['movement_state'].replace('parking', '0', inplace=True, regex=True)
# Konvertierung von String zu float (str->float; anstatt int, da NaN für den Typ int nicht erlaubt ist)
dfMaschen.movement_state = dfMaschen.movement_state.astype(float)


In [ ]:
# Werte, die außerhalb des Geofences um den Maschen Bahnhof liegen entfernen
# Punkte für ein Polygon um den Maschenbahnhof

# exaktes Polygon um Maschen
'''polygonPunkteMaschen = np.array([
    [53.391718, 10.087218],
    [53.393941, 10.072218],
    [53.397919, 10.059582],
    [53.406176, 10.047843],
    [53.406176, 10.047843],
    [53.424749, 10.027210],
    [53.424608, 10.030620],
    [53.418648, 10.039877],
    [53.414849, 10.048505],
    [53.412179, 10.053845],
    [53.409576, 10.057802],
    [53.404956, 10.064698],
    [53.397338, 10.080605],
    [53.392461, 10.096895],
    [53.388525, 10.096012]
])
'''

# grobes Polygon um Maschen
polygonPunkteMaschen = np.array([
    [53.430275, 10.03927],
    [53.422016, 10.020830],
    [53.398547, 10.035114],
    [53.386725, 10.089851], 
    [53.397499, 10.103144],
    [53.417347, 10.075169]
])

In [ ]:
# polygon erstellen mit den Punkten um den Maschenbahnhof
polygonMaschen = Polygon(polygonPunkteMaschen)
polygonMaschen

In [ ]:
def isCoordinateInMaschen(row):
    """
    Funktion, die für eine Reihe prüft, ob die Koordinaten im Maschenbahnhof liegen oder nicht
    :param row: Reihe eines pandas DataFrame
    :return: Reihe eines pandas DataFrame mit zusätzlicher Spalte (bool), ob Punkt im
             Maschen Bahnhof liegt
    """
    lat = row['latitude']
    long = row['longitude']
    point = Point(lat, long)
    # Überprüfen, ob der Punkt in Maschen liegt
    boolPoint = point.within(polygonMaschen)
    row['is_in_maschen'] = boolPoint
    return row

In [ ]:
print(len(dfMaschen))

# Die Funktion "isCoordinateInMaschen" wird auf den DataFrame angewendet
dfMaschen = dfMaschen.apply(isCoordinateInMaschen, axis=1)

# dfMaschenMapping = dfMaschenMapping[dfMaschenMapping.is_in_maschen]
dfMaschen = dfMaschen[dfMaschen['is_in_maschen']==1]
dfMaschen.reset_index(drop=True, inplace=True)

print(len(dfMaschen))

In [ ]:
import matplotlib.pyplot as plt

# wagonID2Plot 1017603201772777

# Überblick durch Geodaten und the wagon moving state"
wagonIDList = np.sort(dfMaschen["wagon_ID"].drop_duplicates().to_numpy())

from random import randrange
radn_ind = randrange(len(wagonIDList))

if 1:
    wagonID2Plot = wagonIDList[radn_ind]
    print(wagonID2Plot)
else: 
    wagonID2Plot =1017603204104556
    print(wagonID2Plot)
    

map_maschen = plt.imread(os.path.join('..', '..', 'data', 'map.png'))
BBox = (10.0208, 10.1031, 53.3867, 53.4303)


Y = dfMaschen[dfMaschen["wagon_ID"] == wagonID2Plot]
X = Y.sort_values(by=['timestamp_measure_position'])


fig, ax = plt.subplots(ncols=2, figsize = (7,4.666), constrained_layout=True, sharey=True)
ax[0].plot(X['longitude'],
           X['latitude'], color='blue', zorder=1, linewidth=0.5)
ax[0].scatter(X['longitude'],
           X['latitude'], s = 6, c = X['timestamp_measure_position'], cmap='Blues', zorder=2, marker='.')


for axes in ax:
    axes.set_xlim(BBox[0], BBox[1])
    axes.set_ylim(BBox[2], BBox[3])
    axes.imshow(map_maschen, extent= BBox, zorder=0, aspect='equal')
    axes.set_aspect(1.5)
    axes.grid(False)

ax[0].set_xlabel('Longitude (Breitengrad)', fontsize = 15)
ax[1].set_xlabel('Longitude (Breitengrad)', fontsize = 15)

ax[0].set_ylabel('Latitude (Lägengrad)', fontsize = 15)

fig.savefig(os.path.join('..', 'doc', 'wagonID.pdf'))


inDHMS = list(np.ones(len(X)))

# timestamp Sekunden in Tage Stunden etc umrechnen 
import datetime 
  
def convert(n): 
    return str(datetime.timedelta(seconds = n))



for ii, time in enumerate(X['timestamp_measure_position'].values):
    inDHMS[ii] = convert(time)

X.loc[:,'time_position'] = inDHMS
    
print(X[['movement_state','time_position' ]].head(30))

In [ ]:
dfMaschen["wagon_ID"]

## Create list in which in every row a df for every single Ride is stored

List is called: **list_splitRide**

ein _ride_ ist folgend definiert: 
- zwischen zwei Zustandspunkten liegt mindestens ein Zeitabstand von 2 Stunden vor
- der _movement_state_ ist nicht parken (da hier die 2 Stunden regulär überschritten werden dürfen)

In [ ]:
#pd.options.mode.chained_assignment = None  # default='warn'
dfMaschen['wagon_ID'].replace('\'', '', inplace=True, regex=True)
# Strings werden in integer umgewandelt
dfMaschen.wagon_ID = dfMaschen.wagon_ID.astype('int64')     #int64 damit Funktionalität auch für OS = Windows
wagon_IDList = np.sort(dfMaschen["wagon_ID"].drop_duplicates().to_numpy())
dfMaschen_md = pd.DataFrame()
count = 0

for ii, wagon_ID in enumerate(wagon_IDList):
    # df erst nach wagon_ID sortiert, dann jede einzelne wagon_Id nochmals nach der Zeit
    df_ID = dfMaschen[dfMaschen['wagon_ID'] == wagon_ID]
    df_ID_sort = df_ID.sort_values(by=['timestamp_measure_position'])
    
    # einzelnen Wagendaten aufteilen nach Fahrten (einzelne Fahrten in Maschen)
    diff_time = df_ID_sort['timestamp_measure_position'].diff()
    diff_time.drop(index=diff_time.index[0], axis=0, inplace=True) # erster Eintrag löschen, um die Zeitdifferenz um ein Element nach vorne zu schieben
    diff_time = diff_time.append(pd.Series(data=[0])) # damit der Vektor wieder die richtige Länge besitzt
    diff_time = diff_time.reset_index(drop=True)
    
    # fillna(ffill) füllt alle NaN Werte mit dem letzten nicht NaN Werte auf
    # wird bennötig, da Movement State Event Message --> wird nur bei einer Änderung betätigt
    df_ID_sort['movement_state'] = df_ID_sort['movement_state'].fillna(method="ffill")
    df_ID_sort = df_ID_sort.reset_index(drop=True)
    
    # Zeilen finden, bei denen der Zeitabstand größer als 2 Stunden ist (d.h. der Wagon hat Maschen verlassen und ist zu einem späteren Zeitpunkt wieder eingefahren)
    single_ride = np.array(np.ones(len(diff_time))-1)
    single_ride[diff_time > 20*60] = 1 # eine 1 beschreibt, dass dies der letzte Eintrag einer Fahrt ist
    
    
    single_ride[(diff_time > 20*60) & (diff_time < 24.1*60*60) & (df_ID_sort['movement_state'] == 0)] = 0 # da wenn der Wagon parkt auch ein die Zeitdifferenz überschritten würde 
    ind_Ride_end = np.where(single_ride == 1)[0]
    
    # ind_Ride_start: # der Wert enspricht dem Index des ersten Eintrags einer neuen Fahrt
    ind_Ride_start = ind_Ride_end + 1 # ind_Ride gibt ab hier die Position einer neuen Fahrt an
    ind_Ride_start = np.append(np.array(0),ind_Ride_start) # um for_Schleife zu vereinfachen wird erste Position des Vektors auch abgespeichert
    ind_Ride_start = np.append(ind_Ride_start,np.array(len(diff_time))) # die letzte Zahl ist 1 höher als es Positinen gibt, damit pos_movState_0 einfacher ermittelt werden kann

    df_ID_sort['Ride'] = -1
    
    for ll, ind in enumerate(ind_Ride_start):

        if ll < len(ind_Ride_start)-1: # because otherwise ll+1 would cause an error
            df_ID_sort['Ride'][ind:ind_Ride_start[ll+1]] = count
            count += 1
        else: 
            df_ID_sort.loc[['Ride'][ind:]] = count
            count += 1
        #moveCountWagon = np.append(moveCountWagon,movement_count)

    # wieder zu einem großen df zusammenfügen
    dfMaschen_md = dfMaschen_md.append(df_ID_sort)
    
dfMaschen = dfMaschen_md
print(len(dfMaschen))

In [ ]:
import matplotlib.pyplot as plt

# RideList 3335, 3765, 1619, 4839, 3587, 349, 4000, 5168, mit 0.5: 3836, 4925 mit 20min: 7490 4529

# Überblick durch Geodaten und the wagon moving state"
RideList = np.sort(dfMaschen["Ride"].drop_duplicates().to_numpy())

from random import randrange
radn_ind = randrange(len(RideList))

if 0:
    Ride2Plot = RideList[radn_ind]
else: Ride2Plot = 2903

X = dfMaschen[dfMaschen["Ride"] == Ride2Plot]

#X = X[X['signal_quality_hdop']<3]

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,10))
ax.scatter(X['longitude'],
           X['latitude'],
           c = X['timestamp_measure_position'], s = 20)

ax.plot(X['longitude'],
           X['latitude'])

ax.set_xlabel('longitude', fontsize = 15)
ax.set_ylabel('latitude', fontsize = 15)
ax.set_title('Beispielfahrt von Ride '+str(Ride2Plot)+ ' mit gezeigtem movement_count', fontsize = 20)  
ax.scatter(polygonPunkteMaschen[:,1],polygonPunkteMaschen[:,0],marker='x')
plt.show()

inDHMS = list(np.ones(len(X)))

# timestamp Sekunden in Tage Stunden etc umrechnen 
import datetime 
  
def convert(n): 
    return str(datetime.timedelta(seconds = n))



for ii, time in enumerate(X['timestamp_measure_position'].values):
    inDHMS[ii] = convert(time)

X.loc[:,'time_position'] = inDHMS
    
print(X[['wagon_ID','movement_state','time_position' ]].head(30))

In [ ]:
#from collections import Counter
#Counter(dfMaschen['Ride'])
#dfMaschen_md[['Ride','timestamp_measure_position','wagon_ID']].tail(50)
#dfMaschen_md['Ride'].iloc[0]
#ind_Ride_start
#len(diff_time)
#diff_time

#### !!!Überprüfen, ob timestamp_measure_movement_state nahe an timestamp_measure_position

In [ ]:
print(len(dfMaschen))
# Reihen, die beim cleaning von NaN ignoriert werden
cols_to_ignore = ['timestamp_measure_movement_state']
searchInCols = [x for x in dfMaschen.columns.tolist() if x not in cols_to_ignore]
# Reihen mit NaN werden bereinigt (außer die NaN ist in einer der "cols_to_ignore"
dfMaschen.dropna(subset=searchInCols, axis=0)
print(len(dfMaschen))

# Altitude muss eigenständig / anders behandelt werden, da dort Datentyp: "numpy.float64"
# wodurch ein "nan" anstatt "NaN"
### dfMaschen = dfMaschen[dfMaschen['altitude'].notna()]
# Zurücksetzen des Index
dfMaschen.reset_index(drop=True, inplace=True)

# Umwandeln des 'loading_state' und der 'wagon_ID' von Strings in numerische Datentypen
# String Operationen zum ersetzen der Wörter
dfMaschen['loading_state'].replace('Beladen', '1', inplace=True, regex=True)
dfMaschen['loading_state'].replace('Leer', '0', inplace=True, regex=True)

# Strings werden in integer umgewandelt
dfMaschen.loading_state = dfMaschen.loading_state.astype('int64')
# Der Movement State wird erst nach einem späteren Preprocessing umgewandelt
# dfMaschen.movement_state.astype('float')     #use float, because NaN does not exist in integers

print('vorher '+str(len(dfMaschen)))
dfMaschen.drop(dfMaschen.loc[dfMaschen['GNSS_velocity'].isna()].index, inplace=True)
print('nach NaN GNSS_velocity '+str(len(dfMaschen)))
dfMaschen.drop(dfMaschen.loc[dfMaschen['movement_state'].isna()].index, inplace=True)
print('nach NaN movement_state '+str(len(dfMaschen)))

if False:
    # Reihen löschen, in denen der Ladezustand bzw. der Zeitpunkt = 'NaT' sind
    dfMaschen.drop(dfMaschen.loc[dfMaschen['loading_state_update']=='NaT'].index, inplace=True)
    dfMaschen.drop(dfMaschen.loc[dfMaschen['timestamp_transfer']=='NaT'].index, inplace=True)
    dfMaschen.drop(dfMaschen.loc[dfMaschen['timestamp_index']=='NaT'].index, inplace=True)


In [ ]:

# Anwenden der Funktion format_timestamp
dfMaschen['loading_state_update'] = dfMaschen['loading_state_update'].apply(format_timestamp) 
dfMaschen['timestamp_transfer'] = dfMaschen['timestamp_transfer'].apply(format_timestamp) 
dfMaschen['timestamp_index'] = dfMaschen['timestamp_index'].apply(format_timestamp) 
# NaT timestamps als NaT belassen im Falle des 'timestamp_measure_movement_state' --> für späteres Vorgehen relevant
dfMaschen['timestamp_measure_movement_state'] = dfMaschen['timestamp_measure_movement_state'].apply(lambda x: format_timestamp(x) if x != 'NaT' else 'NaT') 

# Verwenenden des "mapping" files um den "wagon_type" und "wagon_construction" hinzuzufügen
# Hinweis: wenn "wagon_ID" nicht im "mapping" file wird die Zeile entfernt (--> keine Zuordnung möglich)
# Einlesen der mapping pickle Datei
mapPath = os.path.join("..", "..", "data", "mappingDf.pickle")
dfMapping = pd.read_pickle(mapPath)
# Alle NaN Werte droppen
# "provider" aus dem "mapping" DataFrame droppen, da "provider" auch schon in dfMaschen vorkommt
print(len(dfMapping))
dfMapping.drop('provider', axis=1, inplace=True)
print(len(dfMapping))
dfMapping.reset_index(drop=True, inplace=True)
# Verbinden der zwei Tabellen: mapping und dfMaschen nach der gemeinsamen Zeile: wagon_ID
dfMaschenMapping = pd.merge(dfMaschen, dfMapping, on = "wagon_ID")
print(len(dfMaschenMapping))

In [ ]:
dfMaschenMapping

### Entfernen von spezifischen Werten
z.B.:
- Geschwindigkeiten von größer > 130 km/h für Güterwagons keine sinnvolle Werte
(insbesondere) da im Bahnhof
- Höhen, die größer sind als 50 m sind nicht realistisch, da Maschen auf 11 m über N.N,
wird aber an dieser Stelle nicht gedroppt, da Altitude aufgrund der hohen Ungenauigkeit und dem geringen (bzw. keinem) Mehrwert nicht verwendet wird
(https://www.mapcoordinates.net/de)

In [ ]:
# Zeilen mit Geschwindigkeiten größer 130 km/h entfernen
dfMaschenMapping.drop(dfMaschenMapping[dfMaschenMapping['GNSS_velocity'] > 130].index, inplace=True)
dfMaschenMapping.reset_index(drop=True, inplace=True)

print(len(dfMaschenMapping))

In [ ]:
# Funktionen zum entfernen von Ausreißern (oberhalb 99% Quantil und unterhalb 1% Quantil)
# Aufspüren der Grenzen von Ausreißern für eine bestimmte Variable in einem DataFrame
def outlierThresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantileRange = quartile3 - quartile1
    upLimit = quartile3 + 1.5 * interquantileRange
    lowLimit = quartile1 - 1.5 * interquantileRange
    return lowLimit, upLimit

# Funktion, bei der Werte ober-/unterhalb der Quantile mit dem entsprechenden
# Quantilwert ersetzt werden
def replaceWithThresholds(dataframe, variable):
    lowLimit, upLimit = outlierThresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < lowLimit), variable] = lowLimit
    dataframe.loc[(dataframe[variable] > upLimit), variable] = upLimit
    
# Funktion, bei der Werte ober-/unterhalb der Quantile gelöscht werden
def removeThresholds(dataframe, variable):
    lowLimit, upLimit = outlierThresholds(dataframe, variable)
    dataframe.drop(dataframe[dataframe[variable] < lowLimit].index, inplace=True)
    dataframe.drop(dataframe[dataframe[variable] > upLimit].index, inplace=True)


In [ ]:
# Anwendung auf sinnvolle Spalten/Variablen
listToRemoveTresholds = ["signal_quality_satellite",
                         "signal_quality_hdop",
                         "GNSS_velocity",           # Anmerkung: muss diskutiert werden
                         ]
# Variablen betrachten mit: dfMaschenMapping.columns

# Achtung: nur einmalige Anwendung!
for var in listToRemoveTresholds:
    removeThresholds(dfMaschenMapping, var)
print(len(dfMaschenMapping))

In [ ]:
dfMaschenMapping.reset_index(drop=True, inplace=True)

In [ ]:
safePath = os.path.join("..", "..", "data", "maschenPickled.pickle")
dfMaschenMapping.to_pickle(safePath)